In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import pandas_profiling
import os
from datetime import datetime, timedelta
import dateutil.relativedelta
import gc

pd.set_option('display.max_columns',None)

import s3fs
import pyarrow.parquet as pq

def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe

def func_activa(base,date,n_mes,nom_col):
    if len(base)>0:
        date_fin = date + relativedelta(months=n_mes)

        data_act_temp = data_act.loc[(data_act['mes_activacion']>=date) & (data_act['mes_activacion']<=date_fin),['cedula','Numero_cuenta','fecha_activacion','mes_activacion']].drop_duplicates()
        ids_mes = data_act_temp['cedula'].unique()
        
        base.loc[base['cedula'].isin(ids_mes),'venta'] = 1    
        base.loc[base['venta'].isnull(),'venta'] = 0
        
        base.loc[:,nom_col] = 1   
        #base = pd.merge(base,data_act_temp,on=['cedula','ID_CLIENTE'],how='left')
        base = pd.merge(base,data_act_temp,on=['cedula'],how='left')
    
    return base

In [2]:
input_paths = {'crm':['s3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-crm_M202103.csv',
                     's3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-crm_H20200801-20210201.csv',
                      's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20200228-20200601.csv',
                     's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190801-20200201.csv',
                     's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190104-20190702.csv'],
              'vendors':['s3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-vendors_H20201201-20210301.csv',
                        's3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-vendors_H20200801-20210101.csv',
                         's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20200401-20200601.csv',
                        's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20190101-20200301.csv']}


obj = 'crm'

files = input_paths[obj]

for ix, file in enumerate(files):
    print('File:', file)
    
    data_temp = pd.read_csv(file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str}) 
    
    if ix == 0:
        data_crm = data_temp
    else:
        data_crm = pd.concat([data_crm,data_temp], ignore_index=True)
        
    del data_temp

File: s3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-crm_M202103.csv
File: s3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-crm_H20200801-20210201.csv
File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20200228-20200601.csv
File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190801-20200201.csv
File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190104-20190702.csv


In [3]:
data_crm.head(5)

,NUM_DOC,ID_CLIENTE,FECHA
0,73076531,NaN,01/03/2021
1,71255478,NaN,01/03/2021
2,79272874,NaN,01/03/2021
3,39446646,NaN,01/03/2021
4,1130598082,NaN,01/03/2021


In [4]:
## tipo campaña
data_crm['TIPO CAMPAÑA'] = 'Preaprobados'
data_crm['tipo_camp'] = 'of-preaprobados'


# Casting variables
data_crm['FECHA'] = pd.to_datetime(data_crm['FECHA'],format='%d/%m/%Y')
data_crm['mes_campaña'] = pd.to_datetime(np.where(data_crm['FECHA'].dt.day>=28, 
                                                              data_crm['FECHA']+timedelta(days=10),
                                                              data_crm['FECHA']),format='%Y-%m-%d')
data_crm['mes_campaña2'] = pd.to_datetime(data_crm['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
data_crm['mes_campaña'] = pd.to_datetime(data_crm['mes_campaña'].dt.strftime('%Y-%m'),format='%Y-%m')

In [5]:
data_crm.head(5)

,NUM_DOC,ID_CLIENTE,FECHA,TIPO CAMPAÑA,tipo_camp,mes_campaña,mes_campaña2
0,73076531,NaN,2021-03-01,Preaprobados,of-preaprobados,2021-03-01,2021-03-01
1,71255478,NaN,2021-03-01,Preaprobados,of-preaprobados,2021-03-01,2021-03-01
2,79272874,NaN,2021-03-01,Preaprobados,of-preaprobados,2021-03-01,2021-03-01
3,39446646,NaN,2021-03-01,Preaprobados,of-preaprobados,2021-03-01,2021-03-01
4,1130598082,NaN,2021-03-01,Preaprobados,of-preaprobados,2021-03-01,2021-03-01


In [6]:
obj = 'vendors'

files = input_paths[obj]

for ix, file in enumerate(files):
    print('File:', file)
    
    data_temp = pd.read_csv(file, sep='|', encoding='ISO8859-1',quoting=1, dtype={'ID_CLIENTE':str,'NUM_DOC':str})
    
    if ix == 1:
        data_temp['FECHA2'] = pd.to_datetime(data_temp['FECHA'],format='%d/%m/%Y')
        data_temp = data_temp[data_temp['FECHA2']<'2020-12-01']
        data_temp = data_temp[['TIP_DOC','NUM_DOC','ID_CLIENTE',' CUPO_APROBADO ','FECHA','TIPO CAMPAÑA']]
    if ix == 0:
        data_vend = data_temp
    else:
        data_vend = pd.concat([data_vend,data_temp], ignore_index=True)
        
    del data_temp

File: s3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-vendors_H20201201-20210301.csv
File: s3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-vendors_H20200801-20210101.csv
File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20200401-20200601.csv
File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20190101-20200301.csv


In [7]:
data_vend.head(5)

,TIP_DOC,NUM_DOC,ID_CLIENTE,CUPO_APROBADO,FECHA,TIPO CAMPAÑA
0,1.0,41654623,NaN,500000,01/12/2020,Preaprobados
1,1.0,29612243,NaN,500000,01/12/2020,Preaprobados
2,1.0,72153077,NaN,1.26e+07,01/12/2020,Preaprobados
3,1.0,91297473,NaN,8.1e+06,01/12/2020,Preaprobados
4,1.0,20325011,NaN,500000,01/12/2020,Preaprobados


In [8]:
data_vend['tipo_camp'] = np.where(data_vend['TIPO CAMPAÑA'] == 'Preaprobados','at-preaprobados','at-perfilados')
data_vend['FECHA'] = pd.to_datetime(data_vend['FECHA'],format='%d/%m/%Y')
data_vend['mes_campaña'] = pd.to_datetime(data_vend['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')


cols1 = ['NUM_DOC', 'ID_CLIENTE', 'FECHA', 'mes_campaña', 'TIPO CAMPAÑA', 'tipo_camp']
cols2 = ['NUM_DOC', 'ID_CLIENTE', 'FECHA', 'mes_campaña', 'TIPO CAMPAÑA', 'tipo_camp', 'CUPO_APROBADO']

data_vend.columns = data_vend.columns.str.strip()
data_camp = pd.concat([data_crm[cols1], data_vend[cols2]], ignore_index=True)[cols2]


data_camp['NUM_DOC'] = data_camp['NUM_DOC'].astype(np.int64)
data_camp['FECHA'] = pd.to_datetime(data_camp['FECHA'])
data_camp['mes_campaña'] = pd.to_datetime(data_camp['mes_campaña'])

/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [9]:
print(data_camp.shape)
data_camp.head(5)

(226768, 7)


,NUM_DOC,ID_CLIENTE,FECHA,mes_campaña,TIPO CAMPAÑA,tipo_camp,CUPO_APROBADO
0,73076531,NaN,2021-03-01,2021-03-01,Preaprobados,of-preaprobados,NaN
1,71255478,NaN,2021-03-01,2021-03-01,Preaprobados,of-preaprobados,NaN
2,79272874,NaN,2021-03-01,2021-03-01,Preaprobados,of-preaprobados,NaN
3,39446646,NaN,2021-03-01,2021-03-01,Preaprobados,of-preaprobados,NaN
4,1130598082,NaN,2021-03-01,2021-03-01,Preaprobados,of-preaprobados,NaN


In [11]:
########### ACTIVACIONES

path_act = ['s3://adl-refined-dev-popular/data_orig/productos/activo-tarjeta-credito-activadas/productos_activos-tarjeta-credito-activadas_H202003-202005.csv',
            's3://adl-refined-dev-popular/data_orig/productos/activo-tarjeta-credito-activadas/productos_activos-tarjeta-credito-activadas_H20180101-20200327.csv'
           ]
path = 's3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/Productos_Activo-Tarjeta-credito-nueva_Masterfile_D'


data_act2 = pd.read_csv(path_act[0], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
data_act2 = data_act2.drop_duplicates()

print(data_act2.shape)


data_act1 = pd.read_csv(path_act[1], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
data_act1 = data_act1.drop_duplicates()

print(data_act1.shape)


## Uniendo bases
data_act2['fecha_activacion'] = data_act2['fecha_activacion'].str.replace('/20','/2020')
data_act1 = pd.concat([data_act2,data_act1], ignore_index=True)

# Casting variables
data_act1['fecha_activacion'] = pd.to_datetime(data_act1['fecha_activacion'],format='%d/%m/%Y')
data_act1['mes_activacion'] = pd.to_datetime(data_act1['fecha_activacion'].dt.strftime('%Y-%m'),format='%Y-%m')

del data_act2

(3157, 6)
(40411, 6)


In [12]:
data_act1.head(5)

,Numero_cuenta,Numero_tarjeta,Identificacion,Tipo_identificacion,fecha_activacion,ID_CLIENTE,mes_activacion
0,NaN,539168******0829,160853,2,2020-03-30,392752305340136601,2020-03-01
1,NaN,420559******1208,160853,2,2020-03-30,392752305340136601,2020-03-01
2,NaN,420559******4257,195017,1,2020-05-06,264252317716356802,2020-05-01
3,NaN,420559******4961,208056,1,2020-05-22,295352314103188701,2020-05-01
4,NaN,539238******8374,208056,1,2020-05-22,295352314103188701,2020-05-01


In [13]:
import boto3
import boto3.session

def _key_existing_size__list(client, bucket, key):
    """return the key's size if it exist, else None"""
    response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    for obj in response.get("Contents", []):
        if obj["Key"] == key:
            return obj["Size"]
        
def _key_files__list(client, bucket, key):
    """return the key's size if it exist, else None"""
    response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    return response.get("Contents", [])

def parquet_exists(s3_url):
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    key = s3_url.split("/")[3:]
    key = "/".join(key)+ "/_SUCCESS"
    size = _key_existing_size__list(s3, bucket, key)
    if size is not None:
        print("File exists:", s3_url)
        return True
    return False

def get_s3_parquets(s3_url):
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    key = s3_url.split("/")[3:]
    key = "/".join(key)
    objects = _key_files__list(s3, bucket, key)
    files = ["s3://" + bucket + "/" + file["Key"][:-9] for file in objects if file["Key"].split("/")[-1]=="_SUCCESS"]
    return files

In [14]:
s3_url = 's3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva'
files = get_s3_parquets(s3_url)


for ix, file in enumerate(files):
    print('File:', file)
    data_temp = spark_read_parquet(file)
    if ix == 0:
        data_act = data_temp
    else:
        data_act = pd.concat([data_act,data_temp], ignore_index=True)
        
    del data_temp

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200511


/home/jupyter/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200512
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200513
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200514
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200518
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200519
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200520
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200704
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200705
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200706
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200707
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200708
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200709
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200830
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200831
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200901
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200902
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200903
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200904
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201028
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201029
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201030
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201031
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201101
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201102
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201226
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201227
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201228
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201229
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201230
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201231
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210222
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210223
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210224
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210225
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210226
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210227
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210420
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210421
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210422
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210423
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210424
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210425
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

In [ ]:
files

In [ ]:
print(data_act.shape)
data_act.head()

In [ ]:
# Casting variables
data_act['fecha_activacion_plastico'] = pd.to_datetime(data_act['fecha_activacion_plastico'])
data_act['mes_activacion'] = pd.to_datetime(data_act['fecha_activacion_plastico'].dt.strftime('%Y-%m'),format='%Y-%m')


## Renombrando variables
data_act1.columns = data_act1.columns.str.lower()
data_act1 = data_act1.rename(columns={'identificacion':'cedula'})
data_act = data_act.rename(columns={'id_cli':'cedula',
                                   'tipo_id_cliente':'tipo_identificacion',
                                   'fecha_activacion_plastico':'fecha_activacion'})
data_camp = data_camp.rename(columns={'NUM_DOC':'cedula'})

data_act = data_act[['numero_cuenta', 'numero_tarjeta', 'cedula',
       'tipo_identificacion', 'fecha_activacion', 'id_cliente',
       'mes_activacion']]

In [ ]:
data_act.head(5)

In [ ]:
## Uniendo bases
data_act = pd.concat([data_act1,data_act], ignore_index=True)
data_act = data_act.drop_duplicates()

del data_act1

In [ ]:
## Cast variables
data_act['id_cliente'] = data_act['id_cliente'].fillna('-9999').astype(np.int64)
data_act['tipo_identificacion'] = data_act['tipo_identificacion'].fillna('-9999').astype(np.int64)
data_act['cedula'] = data_act['cedula'].astype(np.int64)

In [ ]:
data_act.head(5).append(data_act.tail(5))

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

str_dates = ['2019-01', '2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
          '2019-12','2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10',
          '2020-11','2020-12','2021-01','2021-02','2021-03']

n_mes = 3 # numero de meses

for ix, date in enumerate(str_dates):
    print('Fecha:',date)
    date = pd.to_datetime(date,format='%Y-%m')
    nom_col = 'camp_'+date.strftime('%Y%m')
    
    base = data_camp[data_camp['mes_campaña']==date]

    if ix == 0:
        base_final = func_activa(base,date,n_mes,nom_col)
        del base
    else:
        base_temp = func_activa(base,date,n_mes,nom_col)
        base_final = pd.concat([base_final,base_temp],ignore_index=True)
        
        del base, base_temp

In [ ]:
base_final.head(5)

In [ ]:
# Casting variables
base_final = base_final[base_final['cedula'].notnull()]

base_final.rename(columns={' CUPO_APROBADO ':'CUPO_APROBADO'}, inplace=True)
base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].str.strip()
base_final['CUPO_APROBADO'] = [str(x).replace('.','') for x in base_final['CUPO_APROBADO']]
base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].astype('float')

cols = ['camp_201901','camp_201902','camp_201903','camp_201904','camp_201905','camp_201906',
            'camp_201907','camp_201908','camp_201909','camp_201910','camp_201911','camp_201912',
            'camp_202001','camp_202002','camp_202003','camp_202004','camp_202005','camp_202006',
        'camp_202008','camp_202009','camp_202010','camp_202011','camp_202012',
        'camp_202101','camp_202102','camp_202103','venta']
                                                                              
base_final.loc[:,cols] = base_final[cols].fillna(0)

In [ ]:
print(base_final.shape)
base_final.head(5)

In [ ]:
base_final.count()

In [ ]:
# Casting variables
base_final['fecha_activacion'] = pd.to_datetime(base_final['fecha_activacion'], format='%Y-%m%d')
base_final['mes_activacion'] = pd.to_datetime(base_final['mes_activacion'], format='%Y-%m%d')

In [ ]:
base_final.head(5)

In [ ]:
pd.crosstab(base_final['mes_campaña'],base_final['venta'])

In [ ]:

## Crear variable meses
def func_var_camp_act(gp_temp, ids):
    
    #producing the first dataframe
    base_out = pd.DataFrame(index=ids) # this is different from the actual prod code
    base_out['mes_campaña'] = date.strftime('%Y-%m-%d')
    
    ## Calculando días primer uso
    base_out['mes_ult_camp'] =  gp_temp['mes_campaña'].max()
    base_out['mes_ult_act'] =  gp_temp['fecha_activacion'].max()
    base_out.loc[base_out['mes_ult_camp'].notnull(),'num_meses_ult_camp'] = (date - base_out.loc[base_out['mes_ult_camp'].notnull(),'mes_ult_camp'])/pd.offsets.Day(30)
    base_out.loc[base_out['mes_ult_act'].notnull(),'num_meses_ult_actv'] = (date - base_out.loc[base_out['mes_ult_act'].notnull(),'mes_ult_act'])/pd.offsets.Day(30)
    
    base_out['num_camp_ult_meses'] = gp_temp['mes_campaña'].count()
    base_out['num_act_utl_meses'] = gp_temp['venta'].sum()
    base_out['num_no_aceptado'] = gp_temp['venta'].value_counts().unstack()[0].fillna(0).astype(int)
    base_out['acep_oferta_prev'] = gp_temp['venta'].max()
    
    return base_out

In [ ]:
str_dates = ['2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
          '2019-12','2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10',
          '2020-11','2020-12','2021-01','2021-02','2021-03']


for ix, dates in enumerate(str_dates):
    print('Periodo:', dates)
    date = pd.to_datetime(dates,format='%Y-%m')
    
    ids = list(base_final.loc[base_final['mes_campaña']==date,'cedula'].unique())
    gp_temp = base_final[base_final['mes_campaña']<date].groupby('cedula')
    gp_temp_ap = base_final[(base_final['mes_campaña']<date)&(base_final['CUPO_APROBADO'].notnull())].groupby('cedula')
    
    if ix == 0:
        tabla_final = func_var_camp_act(gp_temp, ids)
        
        if len(gp_temp_ap)>0:
            tabla_final['prom_cupo_aprob'] = gp_temp_ap['CUPO_APROBADO'].mean()
    
        tabla_final.index.rename('cedula', inplace=True)
        tabla_final.reset_index(inplace = True)
    else:
        base_temp = func_var_camp_act(gp_temp, ids)
        
        if len(gp_temp_ap)>0:
            base_temp['prom_cupo_aprob'] = gp_temp_ap['CUPO_APROBADO'].mean()
    
        base_temp.index.rename('cedula', inplace=True)
        base_temp.reset_index(inplace = True)
        
        tabla_final = pd.concat([tabla_final,base_temp], ignore_index=True)
        print(tabla_final.shape)
        del base_temp
    del gp_temp, date

In [ ]:
tabla_final.head(5)

In [ ]:
## Cast index        
tabla_final['cedula'] = tabla_final['cedula'].fillna('-999')
tabla_final['cedula'] = tabla_final['cedula'].astype('int')

tabla_final= tabla_final[['cedula', 'mes_campaña','acep_oferta_prev', 'mes_ult_act',
       'mes_ult_camp', 'num_act_utl_meses', 'num_camp_ult_meses',
       'num_meses_ult_actv', 'num_meses_ult_camp', 'num_no_aceptado',
       'prom_cupo_aprob']]

tabla_final = tabla_final[(tabla_final['cedula'].notnull()) & (tabla_final['mes_campaña'].notnull())]
tabla_final = tabla_final.drop_duplicates()

tabla_final['data_camp'] = pd.to_datetime(tabla_final['mes_campaña'], format='%Y-%m-%d').dt.strftime('%Y%m').astype('int')

In [ ]:
tabla_final.count()

In [ ]:
tabla_final.head(5)

In [ ]:
tabla_final.head()

In [ ]:
base_final[base_final['cedula']==23132562]

In [ ]:
tabla_final[tabla_final['cedula']==23132562]

In [105]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import pandas_profiling
import os
from datetime import datetime, timedelta
import dateutil.relativedelta
import gc
from datetime import datetime
from dateutil.relativedelta import relativedelta

import boto3
import boto3.session

pd.set_option('display.max_columns',None)

import s3fs
import pyarrow.parquet as pq


def process_camp_activos(input_paths_activos):
    """Pending docstring"""
    
    # Pre_crm. from different route
    data_crm1 = get_crm_new_file(input_paths)
    print('get_crm_new_file')
    # Pre_vend. from different route
    data_vend1 = get_vend_new_file(input_paths)
    print('get_vend_new_file')
    
    # 1. Get crm file
    data_crm = get_crm_file(input_paths)
    print('get_crm_file')
    # 2. Deletes duplicates and casts variables
    data_crm = get_crm_info(data_crm, data_crm1)
    print('get_crm_info')
    print(data_crm.head())
    
    # 3. Get vend file
    data_vend = get_vend_file(input_paths)
    print('get_vend_file')
    # 4. Deletes duplicates, casts variables and union data_crm - data_vend
    data_camp = get_vend_info(data_vend, data_crm, data_vend1)
    print('get_vend_info')
    print(data_camp.head())
    
    # 5. Get activaciones file
    data_act1 = get_activaciones_file(input_paths)
    print('get_activaciones_file')
    # 6. Get activaciones_diarias file
    data_act = get_acti_diarias_file(input_paths)
    print('get_acti_diarias_file')
    # 7. Deletes duplicates and casts variables
    data_act = get_act_diarias_info(data_act, data_act1)
    print('get_act_diarias_info')
    
    # 8. Creates aux variables requeried for the final explanatory variables
    base_final = create_func_activa(data_camp)
    print('create_func_activa')
    # 9. Deletes duplicates and casts variables
    base_final = get_func_activa_info(base_final)
    print('get_func_activa_info')
    
    # 10. Creates aux variables requeried for the final explanatory variables
    tabla_final = create_func_var_camp_act(base_final)
    print('create_func_var_camp_act')
    # 11. Deletes duplicates and casts variables
    tabla_final = get_func_var_camp_act(tabla_final)
    print('get_func_var_camp_act')
    print('camp_activos_df created successfully')  
    
    return tabla_final

#########################################################
################### Bases de campañas ###################
#########################################################

#####Prep. crm_new
def get_crm_new_file(input_paths):
    
    path_file = get_s3_parquet_flag(input_paths['canpañas'], 0)[-1]
    data_temp = pd.read_csv(path_file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str})
    data_temp = data_temp.sort_values(['base_name','date','clientid'])
    data_temp = data_temp.groupby(['clientid','base_name']).first()
    data_temp.reset_index(inplace=True)
    
    ## Filtrar express-la14
    data_temp = data_temp.loc[~data_temp['base_name'].str.contains('EXPRES_LA14')]
    data_temp = data_temp.rename(columns={'clientid':'NUM_DOC'})
    
    ## Oficinas
    data_crm = data_temp.loc[~data_temp['base_name'].str.contains('OFICINA_BOGOTA')]
    ## tipo campaña
    data_crm['TIPO CAMPAÑA'] = 'Preaprobados'
    data_crm['tipo_camp'] = 'of-preaprobados'
    # Casting variables
    data_crm['FECHA'] = pd.to_datetime(data_crm['date'],format='%Y-%m-%d')
    data_crm['mes_campaña'] = pd.to_datetime(np.where(data_crm['FECHA'].dt.day>=28, 
                                                      data_crm['FECHA']+timedelta(days=10),
                                                      data_crm['FECHA']),format='%Y-%m-%d')
    data_crm['mes_campaña2'] = pd.to_datetime(data_crm['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_crm['mes_campaña'] = pd.to_datetime(data_crm['mes_campaña'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_crm1 = data_crm.copy()
    
    return data_crm1


#####Prep. vend_new
def get_vend_new_file(input_paths):
    
    path_file = get_s3_parquet_flag(input_paths['canpañas'], 0)[-1]
    data_temp = pd.read_csv(path_file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str})
    data_temp = data_temp.sort_values(['base_name','date','clientid'])
    data_temp = data_temp.groupby(['clientid','base_name']).first()
    data_temp.reset_index(inplace=True)
    
    ## Filtrar express-la14
    data_temp = data_temp.loc[~data_temp['base_name'].str.contains('EXPRES_LA14')]
    data_temp = data_temp.rename(columns={'clientid':'NUM_DOC'})
    
    data_vend = data_temp.loc[~data_temp['base_name'].str.contains('CALL_ATENTO')]
    # Casting variables
    data_vend['TIPO CAMPAÑA'] = 'Preaprobados'
    data_vend['tipo_camp'] = 'at-preaprobados'
    data_vend['FECHA'] = pd.to_datetime(data_vend['date'],format='%Y-%m-%d')
    data_vend['mes_campaña'] = pd.to_datetime(data_vend['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_vend1 = data_vend.copy()
    
    return data_vend1


#####1. OFICINAS

def get_crm_file(input_paths):
    files = input_paths['crm']
    
    for ix, file in enumerate(files):
        print('File:', file)
        
        data_temp = pd.read_csv(file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str}) 
        
        if ix == 0:
            data_crm = data_temp
        else:
            data_crm = pd.concat([data_crm,data_temp], ignore_index=True)
            
        del data_temp
    
    return data_crm

## 2.    
## tipo campaña
def get_crm_info(data_crm, data_crm1):
    
    data_crm['TIPO CAMPAÑA'] = 'Preaprobados'
    data_crm['tipo_camp'] = 'of-preaprobados'
    
    # Casting variables
    data_crm['FECHA'] = pd.to_datetime(data_crm['FECHA'],format='%d/%m/%Y')
    data_crm['mes_campaña'] = pd.to_datetime(np.where(data_crm['FECHA'].dt.day>=28,
                                                      data_crm['FECHA']+timedelta(days=10), 
                                                      data_crm['FECHA']),format='%Y-%m-%d')
    data_crm['mes_campaña2'] = pd.to_datetime(data_crm['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_crm['mes_campaña'] = pd.to_datetime(data_crm['mes_campaña'].dt.strftime('%Y-%m'),format='%Y-%m')
    
    data_crm = pd.concat([data_crm1, data_crm], ignore_index=True)
    
    return data_crm


##### 3. ATENTO
def get_vend_file(input_paths):

    files = input_paths['vendors']
    
    for ix, file in enumerate(files):
        print('File:', file)
        
        data_temp = pd.read_csv(file, sep='|', encoding='ISO8859-1',quoting=1, dtype={'ID_CLIENTE':str,'NUM_DOC':str})
        if ix == 1:
            data_temp['FECHA2'] = pd.to_datetime(data_temp['FECHA'],format='%d/%m/%Y')
            data_temp = data_temp[data_temp['FECHA2']<'2020-12-01']
            data_temp = data_temp[['TIP_DOC','NUM_DOC','ID_CLIENTE',' CUPO_APROBADO ','FECHA','TIPO CAMPAÑA']]
        if ix == 0:
            data_vend = data_temp
        else:
            data_vend = pd.concat([data_vend,data_temp], ignore_index=True)
            
        del data_temp
    
    return data_vend


#4. Casting variables
def get_vend_info(data_vend, data_crm, data_vend1):
    
    data_vend['tipo_camp'] = np.where(data_vend['TIPO CAMPAÑA'] == 'Preaprobados','at-preaprobados','at-perfilados')
    data_vend['FECHA'] = pd.to_datetime(data_vend['FECHA'],format='%d/%m/%Y')
    data_vend['mes_campaña'] = pd.to_datetime(data_vend['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    
    data_vend = pd.concat([data_vend1,data_vend], ignore_index=True)
    
    cols = ['NUM_DOC', 'FECHA', 'mes_campaña', 'TIPO CAMPAÑA', 'tipo_camp']
    
    data_vend.columns = data_vend.columns.str.strip()
    data_camp = pd.concat([data_crm[cols], data_vend[cols]], ignore_index=True)[cols]
        
    data_camp['NUM_DOC'] = data_camp['NUM_DOC'].astype(np.int64)
    data_camp['FECHA'] = pd.to_datetime(data_camp['FECHA'])
    data_camp['mes_campaña'] = pd.to_datetime(data_camp['mes_campaña'])
    data_camp = data_camp.rename(columns={'NUM_DOC':'cedula'})
    
    return data_camp

    
##5. ACTIVACIONES
def get_activaciones_file(input_paths):

    data_act2 = pd.read_csv(input_paths['activaciones'][0], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
    data_act2 = data_act2.drop_duplicates()
    print(data_act2.shape)
    
    data_act1 = pd.read_csv(input_paths['activaciones'][1], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
    data_act1 = data_act1.drop_duplicates()
    print(data_act1.shape)
    
    # Uniendo bases
    data_act2['fecha_activacion'] = data_act2['fecha_activacion'].str.replace('/20','/2020')
    data_act1 = pd.concat([data_act2,data_act1], ignore_index=True)
    
    # Casting variables
    data_act1['fecha_activacion'] = pd.to_datetime(data_act1['fecha_activacion'],format='%d/%m/%Y')
    data_act1['mes_activacion'] = pd.to_datetime(data_act1['fecha_activacion'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_act1.columns = data_act1.columns.str.lower()
    data_act1 = data_act1.rename(columns={'identificacion':'cedula'})
    del data_act2
    
    return data_act1


##6. bases diarias
def get_acti_diarias_file(input_paths):

    files = get_s3_parquets(input_paths['acti_diarias'][0])
    
    for ix, file in enumerate(files):
        print('File:', file)
        data_temp = spark_read_parquet(file)
        if ix == 0:
            data_act = data_temp
        else:
            data_act = pd.concat([data_act,data_temp], ignore_index=True)
            
        del data_temp
        
    return data_act

        
#7. Casting variables     
def get_act_diarias_info(data_act, data_act1):

    data_act['fecha_activacion_plastico'] = pd.to_datetime(data_act['fecha_activacion_plastico'])
    data_act['mes_activacion'] = pd.to_datetime(data_act['fecha_activacion_plastico'].dt.strftime('%Y-%m'),format='%Y-%m')
    
    ## Renombrando variables
    data_act = data_act.rename(columns={'id_cli':'cedula',
                                       'tipo_id_cliente':'tipo_identificacion',
                                       'fecha_activacion_plastico':'fecha_activacion'})
    
    data_act = data_act[['numero_cuenta', 'numero_tarjeta', 'cedula', 'tipo_identificacion', 'fecha_activacion', 'id_cliente',
           'mes_activacion']]
    
    ## Uniendo bases
    data_act = pd.concat([data_act1, data_act], ignore_index=True)
    data_act = data_act.drop_duplicates()

    del data_act1

    ## Cast variables
    data_act['id_cliente'] = data_act['id_cliente'].fillna('-9999').astype(np.int64)
    data_act['tipo_identificacion'] = data_act['tipo_identificacion'].fillna('-9999').astype(np.int64)
    data_act['cedula'] = data_act['cedula'].astype(np.int64)

    return data_act


###8. func_activa
def create_func_activa(data_camp):

    str_dates = ['2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
                 '2019-12','2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10', 
                 '2020-11','2020-12','2021-01','2021-02','2021-03','2021-04','2021-05']
    n_mes = 3 # numero de meses
    
    for ix, date in enumerate(str_dates):
        print('Fecha:',date)
        date = pd.to_datetime(date,format='%Y-%m')
        nom_col = 'camp_'+date.strftime('%Y%m')
        
        base = data_camp[data_camp['mes_campaña']==date]
    
        if ix == 0:
            base_final = func_activa(base, date, n_mes, nom_col)
            del base
        else:
            base_temp = func_activa(base, date, n_mes, nom_col)
            base_final = pd.concat([base_final, base_temp], ignore_index=True)
            
            del base, base_temp

    return base_final


#9. Casting variables
def get_func_activa_info(base_final):
    
    base_final = base_final[base_final['cedula'].notnull()]
    
    # base_final.rename(columns={' CUPO_APROBADO ':'CUPO_APROBADO'}, inplace=True)
    # base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].str.strip()
    # base_final['CUPO_APROBADO'] = [str(x).replace('.','') for x in base_final['CUPO_APROBADO']]
    # base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].astype('float')
    
    cols = ['camp_201901','camp_201902','camp_201903','camp_201904','camp_201905','camp_201906','camp_201907',
            'camp_201908','camp_201909','camp_201910','camp_201911','camp_201912','camp_202001','camp_202002',
            'camp_202003','camp_202004','camp_202005','camp_202006','camp_202008','camp_202009','camp_202010',
            'camp_202011','camp_202012','camp_202101','camp_202102','camp_202104','camp_202105',
            'venta']
    
    base_final.loc[:,cols] = base_final[cols].fillna(0)
    
    # Casting variables
    base_final['fecha_activacion'] = pd.to_datetime(base_final['fecha_activacion'], format='%Y-%m%d')
    base_final['mes_activacion'] = pd.to_datetime(base_final['mes_activacion'], format='%Y-%m%d')

    return base_final

##10 Crear variable meses
def create_func_var_camp_act(base_final):

    str_dates = ['2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
                 '2019-12','2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09',
                 '2020-10','2020-11','2020-12','2021-01','2021-02','2021-03','2021-04','2021-05']
    
    for ix, dates in enumerate(str_dates):
        print('Periodo:', dates)
        date = pd.to_datetime(dates,format='%Y-%m')
        
        ids = list(base_final.loc[base_final['mes_campaña']==date,'cedula'].unique())
        gp_temp = base_final[base_final['mes_campaña']<date].groupby('cedula')
        # gp_temp_ap = base_final[(base_final['mes_campaña']<date)&(base_final['CUPO_APROBADO'].notnull())].groupby('cedula')
        
        if ix == 0:
            tabla_final = func_var_camp_act(gp_temp, ids, date)
            
        #    if len(gp_temp_ap)>0:
        #        tabla_final['prom_cupo_aprob'] = gp_temp_ap['CUPO_APROBADO'].mean()
        
            tabla_final.index.rename('cedula', inplace=True)
            tabla_final.reset_index(inplace = True)
        else:
            base_temp = func_var_camp_act(gp_temp, ids, date)
            
        #    if len(gp_temp_ap)>0:
        #        base_temp['prom_cupo_aprob'] = gp_temp_ap['CUPO_APROBADO'].mean()
        
            base_temp.index.rename('cedula', inplace=True)
            base_temp.reset_index(inplace = True)
            
            tabla_final = pd.concat([tabla_final,base_temp], ignore_index=True)
            
            del base_temp
        del gp_temp, date
    
    return tabla_final

    
##11. Cast index
def get_func_var_camp_act(tabla_final):

    tabla_final['cedula'] = tabla_final['cedula'].fillna('-999')
    tabla_final['cedula'] = tabla_final['cedula'].astype('int')
    
    tabla_final= tabla_final[['cedula', 'mes_campaña','acep_oferta_prev', 'mes_ult_act',
                              'mes_ult_camp', 'num_act_utl_meses', 'num_camp_ult_meses',
                              'num_meses_ult_actv', 'num_meses_ult_camp', 'num_no_aceptado']]
    
    tabla_final = tabla_final[(tabla_final['cedula'].notnull()) & (tabla_final['mes_campaña'].notnull())]
    tabla_final = tabla_final.drop_duplicates()
    
    tabla_final['data_camp'] = pd.to_datetime(tabla_final['mes_campaña'], format='%Y-%m-%d').dt.strftime('%Y%m').astype('int')
    tabla_final = tabla_final.sort_values(['cedula', 'mes_campaña'])
    
    return tabla_final


######## Funciones utilizadas
    
def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe

def func_activa(base, date, n_mes, nom_col):
    if len(base)>0:
        date_fin = date + relativedelta(months=n_mes)

        data_act_temp = data_act.loc[(data_act['mes_activacion']>=date) & (data_act['mes_activacion']<=date_fin),['cedula','Numero_cuenta','fecha_activacion','mes_activacion']].drop_duplicates()
        ids_mes = data_act_temp['cedula'].unique()
        
        base.loc[base['cedula'].isin(ids_mes),'venta'] = 1    
        base.loc[base['venta'].isnull(),'venta'] = 0
        
        base.loc[:,nom_col] = 1   
        #base = pd.merge(base,data_act_temp,on=['cedula','ID_CLIENTE'],how='left')
        base = pd.merge(base,data_act_temp,on=['cedula'],how='left')
    
    return base

def func_var_camp_act(gp_temp, ids, date):
    
    #producing the first dataframe
    base_out = pd.DataFrame(index=ids) # this is different from the actual prod code
    base_out['mes_campaña'] = date.strftime('%Y-%m-%d')
    
    ## Calculando días primer uso
    base_out['mes_ult_camp'] =  gp_temp['mes_campaña'].max()
    base_out['mes_ult_act'] =  gp_temp['fecha_activacion'].max()
    base_out.loc[base_out['mes_ult_camp'].notnull(),'num_meses_ult_camp'] = (date - base_out.loc[base_out['mes_ult_camp'].notnull(),'mes_ult_camp'])/pd.offsets.Day(30)
    base_out.loc[base_out['mes_ult_act'].notnull(),'num_meses_ult_actv'] = (date - base_out.loc[base_out['mes_ult_act'].notnull(),'mes_ult_act'])/pd.offsets.Day(30)
    
    base_out['num_camp_ult_meses'] = gp_temp['mes_campaña'].count()
    base_out['num_act_utl_meses'] = gp_temp['venta'].sum()
    base_out['num_no_aceptado'] = gp_temp['venta'].value_counts().unstack()[0].fillna(0).astype(int)
    base_out['acep_oferta_prev'] = gp_temp['venta'].max()
    
    return base_out

def _key_existing_size__list(client, bucket, key):
    """return the key's size if it exist, else None"""
    response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    for obj in response.get("Contents", []):
        if obj["Key"] == key:
            return obj["Size"]
        
def _key_files__list(client, bucket, key):
    """return the key's size if it exist, else None"""
    response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    return response.get("Contents", [])

def parquet_exists(s3_url):
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    key = s3_url.split("/")[3:]
    key = "/".join(key)+ "/_SUCCESS"
    size = _key_existing_size__list(s3, bucket, key)
    if size is not None:
        print("File exists:", s3_url)
        return True
    return False

def get_s3_parquets(s3_url):
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    key = s3_url.split("/")[3:]
    key = "/".join(key)
    objects = _key_files__list(s3, bucket, key)
    files = ["s3://" + bucket + "/" + file["Key"][:-9] for file in objects if file["Key"].split("/")[-1]=="_SUCCESS"]
    return files

def get_s3_parquet_flag(s3_url, flag):
    """
    s3_url: path donde se desea buscar
    flag: permite identificar por cual camino entrar: 1: buscar archivo especifico, 0: buscar carpeta, ejemplo las std
    """
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    print('bucket_last_file: ', bucket)
    key = s3_url.split("/")[3:]
    key = "/".join(key)
    print('key_last_file: ', key)
    files = _key_files__list(s3, bucket, key)
    files = ["s3://" + bucket + "/" + file["Key"] for file in files]
    return files


In [106]:
input_paths_activos = { 'libranzas': ['s3://data-bpop-dev-sandbox/estandarizado/productos/libranzas/productos_libranzas_dwh_M202101'],  ## Mensual 1 mes atras respecto al mes que esta corriendo - coger v2
         'clientes': ['s3://data-bpop-dev-sandbox/estandarizado/clientes/identificacion/clientes_identificacion_mdm_D20210218'],  ## Mensual 1 mes atras - coger v2 #### NO SE NECESITA QUITAR

         'activo': ['s3://data-bpop-dev-sandbox/estandarizado/productos/activo/productos_activo_sarc_M202104'],  ## Mensual 2 meses atras - coger v2

         'canpañas': ['s3://data-bpop-dev-refined/gestion-campanas/preaprobado-tc/'],
         'crm': ['s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20200228-20200601.csv',  ## Quemado
                 's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190801-20200201.csv',  ## Quemado
                 's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190104-20190702.csv'],  ## Quemado
         'vendors':['s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20200401-20200601.csv',  ## Quemado
                    's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20190101-20200301.csv'],  ## Quemado

         'activaciones': ['s3://adl-refined-dev-popular/data_orig/productos/activo-tarjeta-credito-activadas/productos_activos-tarjeta-credito-activadas_H202003-202005.csv',  ## Quemado
                          's3://adl-refined-dev-popular/data_orig/productos/activo-tarjeta-credito-activadas/productos_activos-tarjeta-credito-activadas_H20180101-20200327.csv'],  ## Quemado
         'acti_diarias': ['s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva'],  ##TodosDiarios

         'buro': ['s3://data-bpop-dev-sandbox/estandarizado/riesgos-creditos/buro-comportamiento-financiero'],  ##TodosDiarios

         'libraznas': ['s3://data-bpop-dev-sandbox/estandarizado/productos/libranzas/productos_libranzas_dwh_M202105'],  ## Mensual caida 1 mes atras


         'pasivo': ['s3://data-bpop-dev-sandbox/estandarizado/productos/pasivo/productos_pasivo_aaatr_M202104'],  ## Mensual caida 1 mes atras
}

In [107]:
base_final = process_camp_activos(input_paths_activos)

bucket_last_file:  data-bpop-dev-refined
key_last_file:  gestion-campanas/preaprobado-tc/


/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

get_crm_new_file
bucket_last_file:  data-bpop-dev-refined
key_last_file:  gestion-campanas/preaprobado-tc/


/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

get_vend_new_file
File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20200228-20200601.csv
File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190801-20200201.csv
File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190104-20190702.csv
get_crm_file


...

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201226
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201227
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201228
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201229
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201230
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201231
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210222
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210223
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210224
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210225
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210226
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210227
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210420
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210421
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210422
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210423
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210424
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210425
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:309: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Fecha: 2019-03
Fecha: 2019-04
Fecha: 2019-05
Fecha: 2019-06
Fecha: 2019-07
Fecha: 2019-08
Fecha: 2019-09
Fecha: 2019-10
Fecha: 2019-11
Fecha: 2019-12
Fecha: 2020-01
Fecha: 2020-02
Fecha: 2020-03
Fecha: 2020-04
Fecha: 2020-05
Fecha: 2020-06
Fecha: 2020-07
Fecha: 2020-08
Fecha: 2020-09
Fecha: 2020-10
Fecha: 2020-11
Fecha: 2020-12
Fecha: 2021-01
Fecha: 2021-02
Fecha: 2021-03
Fecha: 2021-04
Fecha: 2021-05
create_func_activa
get_func_activa_info
Periodo: 2019-02
Periodo: 2019-03
Periodo: 2019-04
Periodo: 2019-05
Periodo: 2019-06
Periodo: 2019-07
Periodo: 2019-08
Periodo: 2019-09
Periodo: 2019-10
Periodo: 2019-11
Periodo: 2019-12
Periodo: 2020-01
Periodo: 2020-02
Periodo: 2020-03
Periodo: 2020-04
Periodo: 2020-05
Periodo: 2020-06
Periodo: 2020-07
Periodo: 2020-08
Periodo: 2020-09
Periodo: 2020-10
Periodo: 2020-11
Periodo: 2020-12
Periodo: 2021-01
Periodo: 2021-02
Periodo: 2021-03
Periodo: 2021-04
Periodo: 2021-05
create_func_var_camp_act
get_func_var_camp_act
camp_activos_df created successf

In [108]:
base_final.head()

,cedula,mes_campaña,acep_oferta_prev,mes_ult_act,mes_ult_camp,num_act_utl_meses,num_camp_ult_meses,num_meses_ult_actv,num_meses_ult_camp,num_no_aceptado,data_camp
0,26547752,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,201902
1,28485549,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,201902
2,14165037,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,201902
3,1045505095,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,201902
4,17125223,2019-02-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,201902


In [109]:
tabla_final.shape

(255507, 11)

In [99]:
base_final = base_final[base_final['cedula'].notnull()]

# base_final.rename(columns={' CUPO_APROBADO ':'CUPO_APROBADO'}, inplace=True)
# base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].str.strip()
# base_final['CUPO_APROBADO'] = [str(x).replace('.','') for x in base_final['CUPO_APROBADO']]
# base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].astype('float')

cols = ['camp_201901','camp_201902','camp_201903','camp_201904','camp_201905','camp_201906','camp_201907',
        'camp_201908','camp_201909','camp_201910','camp_201911','camp_201912','camp_202001','camp_202002',
        'camp_202003','camp_202004','camp_202005','camp_202006','camp_202008','camp_202009','camp_202010',
        'camp_202011','camp_202012','camp_202101','camp_202102','camp_202104','camp_202105',
        'venta']

base_final.loc[:,cols] = base_final[cols].fillna(0)

# Casting variables
base_final['fecha_activacion'] = pd.to_datetime(base_final['fecha_activacion'], format='%Y-%m%d')
base_final['mes_activacion'] = pd.to_datetime(base_final['mes_activacion'], format='%Y-%m%d')

In [100]:
str_dates = ['2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
             '2019-12','2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09',
             '2020-10','2020-11','2020-12','2021-01','2021-02','2021-03','2021-04','2021-05']

for ix, dates in enumerate(str_dates):
    print('Periodo:', dates)
    date = pd.to_datetime(dates,format='%Y-%m')
    
    ids = list(base_final.loc[base_final['mes_campaña']==date,'cedula'].unique())
    gp_temp = base_final[base_final['mes_campaña']<date].groupby('cedula')
    # gp_temp_ap = base_final[(base_final['mes_campaña']<date)&(base_final['CUPO_APROBADO'].notnull())].groupby('cedula')
    
    if ix == 0:
        tabla_final = func_var_camp_act(gp_temp, ids, date)
        
    #    if len(gp_temp_ap)>0:
    #        tabla_final['prom_cupo_aprob'] = gp_temp_ap['CUPO_APROBADO'].mean()
    
        tabla_final.index.rename('cedula', inplace=True)
        tabla_final.reset_index(inplace = True)
    else:
        base_temp = func_var_camp_act(gp_temp, ids, date)
        
    #    if len(gp_temp_ap)>0:
    #        base_temp['prom_cupo_aprob'] = gp_temp_ap['CUPO_APROBADO'].mean()
    
        base_temp.index.rename('cedula', inplace=True)
        base_temp.reset_index(inplace = True)
        
        tabla_final = pd.concat([tabla_final,base_temp], ignore_index=True)
        
        del base_temp
    del gp_temp, date

Periodo: 2019-02
Periodo: 2019-03
Periodo: 2019-04
Periodo: 2019-05
Periodo: 2019-06
Periodo: 2019-07
Periodo: 2019-08
Periodo: 2019-09
Periodo: 2019-10
Periodo: 2019-11
Periodo: 2019-12
Periodo: 2020-01
Periodo: 2020-02
Periodo: 2020-03
Periodo: 2020-04
Periodo: 2020-05
Periodo: 2020-06
Periodo: 2020-07
Periodo: 2020-08
Periodo: 2020-09
Periodo: 2020-10
Periodo: 2020-11
Periodo: 2020-12
Periodo: 2021-01
Periodo: 2021-02
Periodo: 2021-03
Periodo: 2021-04
Periodo: 2021-05


In [101]:
tabla_final['cedula'] = tabla_final['cedula'].fillna('-999')
tabla_final['cedula'] = tabla_final['cedula'].astype('int')

tabla_final= tabla_final[['cedula', 'mes_campaña','acep_oferta_prev', 'mes_ult_act',
                          'mes_ult_camp', 'num_act_utl_meses', 'num_camp_ult_meses',
                          'num_meses_ult_actv', 'num_meses_ult_camp', 'num_no_aceptado']]

tabla_final = tabla_final[(tabla_final['cedula'].notnull()) & (tabla_final['mes_campaña'].notnull())]
tabla_final = tabla_final.drop_duplicates()

tabla_final['data_camp'] = pd.to_datetime(tabla_final['mes_campaña'], format='%Y-%m-%d').dt.strftime('%Y%m').astype('int')